# Image Classification using Flow from Directory

In [3]:
# Import numpy for array operations
import numpy as np
import os
os.getcwd()

'/nfsroot/data/home/2237B46'

### Model Architecture

### Define train and validation generators
Loading images for train and test using ImageDataGenerator. 

In [8]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 32

# Train Data Generator
train_datagen = ImageDataGenerator(rescale=1./255)

# Test Data generator
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of '/train'
train_generator = train_datagen.flow_from_directory(
        'train/',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for test data
test_generator = test_datagen.flow_from_directory(
        'test/',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

Found 6634 images belonging to 22 classes.
Found 2847 images belonging to 22 classes.


In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(22, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Train the model

In [11]:
# fit_generator is similar to 'fit'. But instead of x_train and y_train, we pass 'train_generator' 
# which already has the samples and their corresponding target information.

# 'step' here is one mini-batch
# 'steps_per_epoch' is number of batches per epoch.
# Typically 'steps_per_epoch' should be total_samples divided by batch_size
hist = model.fit_generator(
        train_generator,
        steps_per_epoch=6634//batch_size, # '//' in python returns only the quotient
        epochs=4,
        validation_data=test_generator,
        validation_steps=2847//batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/4
207/207 [==============================] - 525s - loss: 2.3749 - acc: 0.4908 - val_loss: 1.5473 - val_acc: 0.6067
Epoch 2/4
207/207 [==============================] - 492s - loss: 1.7109 - acc: 0.5906 - val_loss: 1.4917 - val_acc: 0.6110
Epoch 3/4
207/207 [==============================] - 502s - loss: 1.5092 - acc: 0.6104 - val_loss: 1.5247 - val_acc: 0.6277
Epoch 4/4
207/207 [==============================] - 493s - loss: 1.3618 - acc: 0.6382 - val_loss: 1.4299 - val_acc: 0.6274


In [7]:
# print(train_generator.filenames)
print(train_generator.class_indices) # This returns class labels of directories

{'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4}


### Test the model

In [8]:
test_generator = test_datagen.flow_from_directory(
        'data/test/',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 43 images belonging to 1 classes.


In [9]:
test_prob = model.predict_generator(test_generator, steps=2) # this returns the probabilities
test_pred_classes = np.argmax(test_prob, axis=1) # convert probabilities to classes
print(test_pred_classes)

[2 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 1 1 1 1
 1 1 1 1 1 1]


In [10]:
# Check the corresponding filenames of the predictions
print(test_generator.filenames[:5])

['test/img_100312.jpg', 'test/img_91724.jpg', 'test/img_91729.jpg', 'test/img_91761.jpg', 'test/img_91769.jpg']


### Save test predictions

In [11]:
import pandas as pd 
test_predictions = pd.DataFrame(data=np.vstack((test_generator.filenames, test_pred_classes)).T, 
                                columns=['file', 'label'])

test_predictions.to_csv('test_submission.csv', header=True, sep=',')